<a href="https://colab.research.google.com/github/Solventerritory/FarmFederate-Advisor/blob/feature%2Fdummy-sensor-data-clean/dummy-sensor-data-clean/backend/FarmFederate_Training_Colab_Updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 📋 Step 1: Check GPU & System Info

In [1]:
import torch
import psutil

print("🔍 Checking GPU...")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"✅ GPU Available: {gpu_name}")
    print(f"   CUDA Version: {torch.version.cuda}")
    print(f"   PyTorch Version: {torch.__version__}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

    if 'A100' in gpu_name or 'V100' in gpu_name:
        print("\n⚡ High-end GPU detected! Estimated time: 4-8 hours")
    elif 'T4' in gpu_name:
        print("\n⏰ T4 GPU detected. Estimated time: 10-12 hours")
    else:
        print(f"\n⏰ {gpu_name} detected. Estimated time: 8-10 hours")
else:
    print("❌ No GPU available!")
    print("   Please enable GPU: Runtime > Change runtime type > T4 GPU")
    raise RuntimeError("GPU required for training")

ram_gb = psutil.virtual_memory().total / 1e9
print(f"\n💾 RAM Available: {ram_gb:.2f} GB")
if ram_gb < 12:
    print("⚠️ WARNING: Low RAM. Training may be slower.")

🔍 Checking GPU...
✅ GPU Available: Tesla T4
   CUDA Version: 12.6
   PyTorch Version: 2.9.0+cu126
   GPU Memory: 15.83 GB

⏰ T4 GPU detected. Estimated time: 10-12 hours

💾 RAM Available: 13.61 GB


## 📦 Step 2: Install Dependencies

In [2]:
%%capture
# Install all required packages (output suppressed for cleaner display)
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q transformers>=4.35.0
!pip install -q datasets>=2.14.0
!pip install -q scikit-learn>=1.3.0
!pip install -q matplotlib seaborn pandas numpy
!pip install -q pillow>=10.0.0
!pip install -q timm>=0.9.0
!pip install -q accelerate>=0.24.0

print("✅ All dependencies installed successfully!")

## 📂 Step 3: Clone Repository

In [11]:
import os

# First, navigate to a safe directory
os.chdir('/content')
print(f"Current directory: {os.getcwd()}")

# Remove old directory if exists
!rm -rf FarmFederate-Advisor

# Clone repository
print("\n📥 Cloning repository...")
!git clone https://github.com/Solventerritory/FarmFederate-Advisor.git

# Switch to the correct branch
print("🔄 Switching to updated branch...")
os.chdir('/content/FarmFederate-Advisor')
!git checkout feature/dummy-sensor-data-clean

# Navigate to backend directory
os.chdir('/content/FarmFederate-Advisor/backend')
print(f"\n📂 Working directory: {os.getcwd()}")

# Verify training script exists
import os.path
if os.path.isfile('federated_complete_training.py'):
    print("✅ Training script found!")
    # Check model count
    with open('federated_complete_training.py', 'r') as f:
        content = f.read()
        vlm_count = content.count('"clip-vit-') + content.count('"blip-') + content.count('"bridgetower"') + content.count('"altclip"') + content.count('"chinese-clip"') + content.count('"groupvit"')
        print(f"✅ VLM models found in code: {vlm_count}")
else:
    print("❌ ERROR: Training script not found!")

Current directory: /content

📥 Cloning repository...
Cloning into 'FarmFederate-Advisor'...
remote: Enumerating objects: 2140, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 2140 (delta 12), reused 24 (delta 9), pack-reused 2108 (from 1)
Receiving objects: 100% (2140/2140), 182.90 MiB | 31.66 MiB/s, done.
Resolving deltas: 100% (367/367), done.
Filtering content: 100% (4/4), 3.15 GiB | 41.03 MiB/s, done.
🔄 Switching to updated branch...
Updating files: 100% (1433/1433), done.
Filtering content: 100% (4/4), 1.00 GiB | 85.26 MiB/s, done.
Branch 'feature/dummy-sensor-data-clean' set up to track remote branch 'feature/dummy-sensor-data-clean' from 'origin'.
Switched to a new branch 'feature/dummy-sensor-data-clean'

📂 Working directory: /content/FarmFederate-Advisor/backend
✅ Training script found!
✅ VLM models found in code: 10


## 💾 Step 4: Mount Google Drive (Optional - Recommended)

**Why mount Drive?**
- Saves results even if Colab disconnects
- Easy access to plots and metrics
- Can resume training if interrupted

In [4]:
from google.colab import drive

try:
    drive.mount('/content/drive')

    # Create output directories
    output_dir = '/content/drive/MyDrive/FarmFederate_Results'
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(f"{output_dir}/checkpoints", exist_ok=True)
    os.makedirs(f"{output_dir}/plots", exist_ok=True)
    os.makedirs(f"{output_dir}/results", exist_ok=True)

    print(f"✅ Google Drive mounted successfully")
    print(f"📁 Results will be saved to: {output_dir}")

    DRIVE_MOUNTED = True
except Exception as e:
    print(f"⚠️ Could not mount Drive: {e}")
    print("   Training will continue, but results won't be backed up to Drive")
    DRIVE_MOUNTED = False

Mounted at /content/drive
✅ Google Drive mounted successfully
📁 Results will be saved to: /content/drive/MyDrive/FarmFederate_Results


## 🚀 Step 5: Start Complete Training

**This will train:**
- 13 LLM models (T5, GPT-2, BERT, RoBERTa, ALBERT, DistilBERT, XLNet, ELECTRA, DeBERTa)
- 13 ViT models (ViT, DeiT, Swin, BEiT, ConvNeXt, ResNet, EfficientNet, RegNet, MobileNet)
- 9 VLM models (CLIP-Base, CLIP-Large, BLIP-Base, BLIP2-OPT, BLIP2-Flan-T5, BridgeTower, AltCLIP, Chinese-CLIP, GroupViT)

**Each model trains in:**
1. **Federated mode** (5 clients, 10 rounds)
2. **Centralized mode** (10 epochs)

**Auto-saves after each model completes!**

---

**⚠️ Training will take 6-12 hours depending on GPU.**

**If interrupted:** Just re-run this cell - training will automatically resume from last checkpoint!

In [ ]:
print("="*80)
print("🚀 STARTING COMPLETE TRAINING PIPELINE")
print("="*80)
print("\n📊 Training Configuration:")
print("   - 28 models (13 LLM + 13 ViT + 2 VLM)")
print("   - 56 total runs (federated + centralized)")
print("   - Auto-checkpoint after each model")
print("   - Results saved continuously")
print("\n⏱️ This will take several hours. Progress is logged below.")
print("   You can check results/ folder for completed models.\n")
print("="*80)

# Run the complete training script
!python federated_complete_training.py

print("\n" + "="*80)
print("✅ TRAINING PIPELINE COMPLETED!")
print("="*80)

🚀 STARTING COMPLETE TRAINING PIPELINE

📊 Training Configuration:
   - 28 models (13 LLM + 13 ViT + 2 VLM)
   - 56 total runs (federated + centralized)
   - Auto-checkpoint after each model
   - Results saved continuously

⏱️ This will take several hours. Progress is logged below.
   You can check results/ folder for completed models.

2026-01-10 19:20:51.382044: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768072851.626182    3640 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768072851.695094    3640 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768072852.185414    3640 computation_placer.cc:177] computation placer already register

## 📊 Step 6: Copy Results to Google Drive

In [6]:
if DRIVE_MOUNTED:
    print("📤 Copying results to Google Drive...\n")

    # Copy results
    !cp -r ../results/* /content/drive/MyDrive/FarmFederate_Results/results/ 2>/dev/null
    print("✅ Results copied")

    # Copy plots
    !cp -r ../plots/* /content/drive/MyDrive/FarmFederate_Results/plots/ 2>/dev/null
    print("✅ Plots copied")

    # Copy checkpoints
    !cp -r ../checkpoints/* /content/drive/MyDrive/FarmFederate_Results/checkpoints/ 2>/dev/null
    print("✅ Checkpoints copied")

    print("\n📁 All results saved to: /content/drive/MyDrive/FarmFederate_Results/")
    print("   You can access these files from your Google Drive!")
else:
    print("⚠️ Drive not mounted. Results are in local Colab storage:")
    print("   - /content/FarmFederate-Advisor/results/")
    print("   - /content/FarmFederate-Advisor/plots/")
    print("   - /content/FarmFederate-Advisor/checkpoints/")

📤 Copying results to Google Drive...

✅ Results copied
✅ Plots copied
✅ Checkpoints copied

📁 All results saved to: /content/drive/MyDrive/FarmFederate_Results/
   You can access these files from your Google Drive!


## 📈 Step 7: View Training Results Summary

In [7]:
import json
from IPython.display import display, Image, HTML

# Load results
try:
    with open('../results/all_results.json', 'r') as f:
        results = json.load(f)

    print("="*80)
    print("📊 TRAINING RESULTS SUMMARY")
    print("="*80)
    print(f"\n✅ Successfully trained {len(results)} models\n")

    # Separate federated and centralized
    fed_results = [r for r in results if 'centralized' not in r['model_name']]
    cent_results = [r for r in results if 'centralized' in r['model_name']]

    # Top 5 models
    print("🏆 TOP 5 FEDERATED MODELS:")
    print("-" * 80)
    sorted_fed = sorted(fed_results, key=lambda x: x['final_metrics']['f1_macro'], reverse=True)
    for i, r in enumerate(sorted_fed[:5], 1):
        model_type = r['config']['model_type'].upper()
        print(f"{i}. {r['config']['name']:30} ({model_type:3}) - F1: {r['final_metrics']['f1_macro']:.4f}, Acc: {r['final_metrics']['accuracy']:.4f}")

    print("\n🏆 TOP 5 CENTRALIZED MODELS:")
    print("-" * 80)
    sorted_cent = sorted(cent_results, key=lambda x: x['final_metrics']['f1_macro'], reverse=True)
    for i, r in enumerate(sorted_cent[:5], 1):
        model_type = r['config']['model_type'].upper()
        base_name = r['config']['name']
        print(f"{i}. {base_name:30} ({model_type:3}) - F1: {r['final_metrics']['f1_macro']:.4f}, Acc: {r['final_metrics']['accuracy']:.4f}")

    # Overall comparison
    avg_fed_f1 = sum(r['final_metrics']['f1_macro'] for r in fed_results) / len(fed_results)
    avg_cent_f1 = sum(r['final_metrics']['f1_macro'] for r in cent_results) / len(cent_results)

    print("\n📊 OVERALL COMPARISON:")
    print("-" * 80)
    print(f"Average Federated F1:    {avg_fed_f1:.4f}")
    print(f"Average Centralized F1:  {avg_cent_f1:.4f}")
    print(f"Difference:              {avg_fed_f1 - avg_cent_f1:+.4f}")

    if avg_fed_f1 > avg_cent_f1:
        print("\n✨ Federated learning outperforms centralized on average!")
    else:
        gap = (avg_cent_f1 - avg_fed_f1) / avg_cent_f1 * 100
        print(f"\n📉 Federated is {gap:.2f}% behind centralized (privacy-accuracy tradeoff)")

    print("\n" + "="*80)

except FileNotFoundError:
    print("❌ Results file not found. Make sure training completed successfully.")

❌ Results file not found. Make sure training completed successfully.


## 📊 Step 8: Display Comparison Plots

In [8]:
import glob
from IPython.display import Image, display

print("📊 Displaying comparison plots...\n")

# Get all plot files
plot_files = sorted(glob.glob('../plots/*.png'))

if plot_files:
    print(f"Found {len(plot_files)} plots\n")

    # Display first 10 plots
    for i, plot_path in enumerate(plot_files[:10], 1):
        plot_name = plot_path.split('/')[-1]
        print(f"\n{'='*80}")
        print(f"Plot {i}: {plot_name}")
        print('='*80)
        display(Image(filename=plot_path, width=1000))

    if len(plot_files) > 10:
        print(f"\n... and {len(plot_files) - 10} more plots available in ../plots/")
else:
    print("❌ No plots found. Check if training completed successfully.")

📊 Displaying comparison plots...

❌ No plots found. Check if training completed successfully.


## 📥 Step 9: Download Results (Optional)

In [9]:
# Zip all results for easy download
print("📦 Creating results archive...\n")

!cd /content/FarmFederate-Advisor && zip -r FarmFederate_Results.zip results plots checkpoints -q

print("✅ Archive created: FarmFederate_Results.zip")
print("\n📥 Downloading...")

from google.colab import files
files.download('/content/FarmFederate-Advisor/FarmFederate_Results.zip')

print("\n✅ Download started! Check your browser downloads.")

📦 Creating results archive...

✅ Archive created: FarmFederate_Results.zip

📥 Downloading...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Download started! Check your browser downloads.


## 🔄 Troubleshooting & Tips

### If Training is Interrupted:
Just re-run Step 5. The system automatically:
- Detects completed models
- Skips them
- Continues from where it stopped

### Out of Memory Error:
The batch sizes are optimized for T4 GPU. If you get OOM errors:
1. Restart runtime
2. Training will resume from last checkpoint

### Check Training Progress:
```python
# Run this in a new cell to check progress
!ls -lh ../results/*.json | wc -l
```

### Access Results from Drive:
After training, access your results at:
https://drive.google.com/drive/MyDrive/FarmFederate_Results